In [ ]:
import pandas as pd
import openai
import json
import os
import re
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

# Define the prompting method and language family
language_family = "english"
method = "direct_inference"
model = "gpt-3.5-turbo"

# Load data
input_file_path = f'JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

def translate(claim, model=model):
   system_prompt = ""


   translate_prompt = f"'{claim}' Translate this claim to English."


   translation = openai.ChatCompletion.create(
       model=model,
       messages=[
           {"role": "system", "content": system_prompt},
           {"role": "user", "content": translate_prompt}
       ],
       temperature=0,
   )
   return translation['choices'][0]['message']['content']

def get_gpt_response(claim, model=model):
    translated_claim = translate(claim)

    system_prompt = ""

    user_prompt = f"'{translated_claim}' Is this claim 'true', 'mostly true', 'half true', 'mostly false', or 'false'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."

    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
    )
    return response['choices'][0]['message']['content']

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Function to clean the output text
def clean_output(output):
    # Remove non-alphanumeric characters except spaces
    cleaned_output = re.sub(r'[^a-zA-Z\s]', '', output)
    return cleaned_output

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    output = get_gpt_response(claim)
    
    # Print the model's output
    print(f"Model Output: {output}")
    
    # Clean the output
    cleaned_output = clean_output(output)
    
    # Extract final answer from the cleaned output
    final_answer = None
    if "final answer true" in cleaned_output.lower():
        final_answer = "true"
    elif "final answer false" in cleaned_output.lower():
        final_answer = "false"
    elif "final answer mostly true" in cleaned_output.lower():
        final_answer = "mostly true"
    elif "final answer mostly false" in cleaned_output.lower():
        final_answer = "mostly false"
    elif "final answer half true" in cleaned_output.lower():
        final_answer = "half true"
    
    # Determine correctness or inconclusiveness
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")

FileNotFoundError: [Errno 2] No such file or directory: 'germanic_data.json'